In [ ]:
!pip install facebook_business
!pip install google-api-python-client
!pip install google-cloud-vision

## FACEBOOK API ##
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.campaign import Campaign
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adset import AdSet

## GOOGLE AUTHENTICATION ##
from google.colab import auth

## GENERAL LIBRARIES ##

import csv
import datetime
import io
import json
import pprint
import requests
import pandas as pd
import json

auth.authenticate_user()

print('Successfully imported Python libraries!')

In [ ]:
df = pd.read_csv('link to google Sheet')
df = df[['API Location name','Line Item Status to apply']]

In [ ]:
access_token = 'Access token for Facebook'
app_secret = 'Secret from Facebook'
app_id = 'Application ID from FB'
id = 'Advertiser ID'
FacebookAdsApi.init(app_id, app_secret, access_token)
my_account = AdAccount('Ad Account')
campaigns = my_account.get_campaigns()

In [6]:
campaign = Campaign('Campaign ID')
adsets = campaign.get_ad_sets(fields=[
    AdSet.Field.name,
    AdSet.Field.id,
    AdSet.Field.daily_budget,
])
adsets_df = pd.DataFrame(adsets)

In [ ]:
adsets_df['name'] = adsets_df['name'].str.split('_').str[5]
adsets_df['name'] = adsets_df['name'].str.split(' Targeting Audience').str[0]

In [ ]:
adsets_df['name'] = adsets_df['name'].astype(str)
df['API Location name'] = df['API Location name'].astype(str)

In [5]:
active_df = df.loc[df['Line Item Status to apply'] == 'ACTIVE']
active_df = adsets_df.merge(active_df,left_on='name',right_on='API Location name')

In [4]:
active_df = df.loc[df['Line Item Status to apply'] == 'ACTIVE']
active_df = adsets_df.merge(active_df,left_on='name',right_on='API Location name')
for id in active_df['id']:
  adset = AdSet(id)
  adset[AdSet.Field.status] = AdSet.Status.active
  adset.remote_update()
  print(id +  ' Activated')


In [7]:
paused_df = df.loc[df['Line Item Status to apply'] == 'PAUSED']
paused_df = adsets_df.merge(paused_df,left_on='name',right_on='API Location name')
for id in paused_df['id']:
  adset = AdSet(id)
  adset[AdSet.Field.status] = AdSet.Status.paused
  adset.remote_update()
  print(id +  ' Paused')
